# Surveys

In [1]:
from collections import Counter

import numpy as np
import pandas as pd

## Loading data

In [2]:
excel_file = pd.ExcelFile('data/Encuestas Lunes de Pascua 2017.xlsx')
survey_df = excel_file.parse(1, names=list(range(20)))

In [3]:
survey_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,MONTERÍA,Luz Mery,Lachasme Jimenes,Estudiante-trabajador,25.0,Primaria,NaN,Montería,Córdoba,NaN,3234554559,villalbabianca@gmail.com,SI,Cada Semana,NO,NO,Otra: Visitar la familia,*Hacer trabajos Academicos. *Otr...,NO,"No sabía que lunes, martes y miercoles Santos ..."
1,NaN,Juan Antonio,Santana Vega,Investigación de tipo cultural,70.0,Profesional,Crr 2a N° 46-16,Montería,Córdoba,7892234,3165815337,juansantave@hotamail.com,SI,Cada Semana,NO,SI,*Estudiar *Leer,*Hacer trabajos Academicos. *Otr...,SI,Salio de viaje
2,NaN,Dora Isabel,Hernández Palma,Docente,31.0,Posgrado,Transv 14c N° 44-69,Montería,Córdoba,NaN,3017871058,doraisabel@hotmail.com,SI,Cada dos meses,NO,NO,Descansar,Hacer trabajos academicos,NO,No tuvo tiempo la semana pasada
3,NaN,Raimundo Enrique,Berrocal Escobar,Anciano,69.0,Posgrado,NaN,Montería,Córdoba,NaN,3015707300,NaN,SI,Cada Semana,SI,SI,Otra: Vagar,Para leer por gusto personal,NO,Otra: Porque me dio la gana
4,NaN,Francisco Javier,Bárcenas Fería,Estudiante,19.0,Profesional,NaN,Montería,Córdoba,7860931,3104990974,franjabafe11@gmail.com,SI,Cada Semana,SI,SI,*Estudiar *Leer ...,*Hacer trabajos academicos ...,SI,*No tuvo tiempo la semana pasada ...


In [4]:
survey_df.shape

(590, 20)

In [5]:
col_names_dict = {
    0: 'SUCURSAL',
    1: 'NOMBRES',
    2: 'APELLIDOS',
    3: 'OCUPACION',
    4: 'EDAD',
    5: 'NIVEL_ESCOLAR_FINALIZADO',
    6: 'DIRECCION',
    7: 'MUNICIPIO',
    8: 'DEPARTAMENTO',
    9: 'TLFN_FIJO',
    10: 'TLFN_CEL',
    11: 'EMAIL',
    12: '¿Es usted usuario frecuente del servicio de prestamo externos de material documental?',
    13: 'Maque en la casilla correspondiente la frecuencia con la que hace uso del servicio de préstamo externo de material documental.',
    14: 'Durante esta Semana Santa ¿Usted hizo uso del servicio de prestamo externo de material documental?',
    15: '¿Sabía usted que la Red de Bibliotecas del Banco de la Republica prestó sus servicios en horario normal durante los dias lunes, martes y miercoles Santos?',
    16: 'Los dias de l Semana Santa, habitualmente, usted los dedica a:',
    17: '¿Los materiales que hoy toma en prestamo los utilizara para?',
    18: 'Los materiales que toma en prestamo, este lunes de Pascua, le servirán para hacer trabajos que debe entregar esta misma semana?',
    19: 'La razón por la que solo hoy se acerca a la Biblioteca a tomar en préstamo material documental es:',
}

In [6]:
male_names_df = pd.read_csv('data/nombres_hombres.csv')
male_names_df.dropna(0, subset=['nombre'], inplace=True)

In [7]:
male_names_df.shape

(25442, 3)

In [8]:
male_names_df.head()

,nombre,frec,edad_media
0,ANTONIO,765138,52.7
1,JOSE,706051,58.3
2,MANUEL,658306,52.5
3,FRANCISCO,572088,54.1
4,JUAN,395559,53.7


In [9]:
female_names_df = pd.read_csv('data/nombres_mujeres.csv')
female_names_df.dropna(0, subset=['nombre'], inplace=True)

In [10]:
female_names_df.shape

(24493, 3)

In [11]:
female_names_df.head()

,nombre,frec,edad_media
0,MARIA CARMEN,681108,51.1
1,MARIA,668112,49.3
2,CARMEN,447130,59.5
3,JOSEFA,325874,64.1
4,ISABEL,296241,54.2


## Transforming data

### Sucursal

In [12]:
last_sucursal = ''
for i, row in survey_df.iterrows():
    if row[0] is not np.nan:
        last_sucursal = row[0]
    else:
        survey_df.loc[i, 0] = last_sucursal

survey_df['SUCURSAL'] = survey_df[0]

In [13]:
survey_df.SUCURSAL.value_counts()

IBAGUE           32
San Andrés       31
Leticia          31
PASTO            30
NEIVA            30
GIRARDOT         30
SINCELEJO        30
IPIALES          30
POPAYÁN          30
TUNJA            30
QUIBDÓ           30
PEREIRA          30
MANIZALES        28
CARTAGENA        28
VILLAVICENCIO    27
VALLEDUPAR       25
Florencia        24
HONDA            18
RIOHACHA         18
BARRANQUILLA     16
SANTA MARTA      14
CÚCUTA           12
MONTERÍA          9
Medellín          7
Name: SUCURSAL, dtype: int64

### Sexo

In [14]:
male_names = [x.strip().lower() for x in male_names_df.nombre.unique()]

In [15]:
female_names = [x.strip().lower() for x in female_names_df.nombre.unique()]

In [16]:
def clean(s):
    return s \
        .strip() \
        .lower() \
        .replace('á', 'a') \
        .replace('é', 'e') \
        .replace('í', 'i') \
        .replace('ó', 'o') \
        .replace('ú', 'u') \

In [17]:
def get_gender(names):
    names = clean(names)
    first = names.split()[0]
    if names in male_names:
        gender = 'male'
    elif names in female_names:
        gender = 'female'
    elif first in male_names:
        gender = 'male'
    elif first in female_names:
        gender = 'female'
    else:
        gender = 'UNKNOWN'
    return gender

survey_df['SEXO'] = survey_df[1].apply(get_gender)

In [18]:
survey_df.SEXO.value_counts()

male       295
female     235
UNKNOWN     60
Name: SEXO, dtype: int64

### Rango edad

In [19]:
def bins_to_labels(bins):
    labels = []
    labels.append('UNKNOWN'.format(bins[1]))
    labels.append('{}-'.format(bins[2]))
    for i in range(2, len(bins) - 2):
        labels.append('{}-{}'.format(bins[i]+1, bins[i+1]))
    labels.append('{}+'.format(bins[-2]+1))
    return labels

bins = [-2, -1, 17, 29, 39, 49, 64, 999]
labels = bins_to_labels(bins)

survey_df['RANGO_EDAD'] = pd.cut(survey_df[4], bins=bins, labels=labels)

In [20]:
survey_df.RANGO_EDAD.value_counts()

18-29      288
30-39      104
17-         83
50-64       48
40-49       35
65+         18
UNKNOWN      0
Name: RANGO_EDAD, dtype: int64

### Nivel escolar finalizado

In [21]:
for i, row in survey_df[
    (survey_df[5] != 'Primaria') &
    (survey_df[5] != 'Secundaria') &
    (survey_df[5] != 'Posgrado') &
    (survey_df[5] != 'Profesional')
].iterrows():
    survey_df.loc[i, 5] = 'UNKNOWN'

survey_df['NIVEL_ESCOLAR_FINALIZADO'] = survey_df[5]

In [22]:
survey_df.NIVEL_ESCOLAR_FINALIZADO.value_counts()

Secundaria     266
Profesional    180
Posgrado        80
Primaria        57
UNKNOWN          7
Name: NIVEL_ESCOLAR_FINALIZADO, dtype: int64

### Departamento

In [23]:
survey_df['DEPARTAMENTO'] = survey_df[7]

In [24]:
survey_df.DEPARTAMENTO.value_counts()

Ibagué                32
San Andrés            30
Quibdó                30
Sincelejo             30
Tunja                 28
Leticia               28
Ipiales               28
Pasto                 27
Villavicencio         27
Popayán               27
Manizales             27
Cartagena             26
Neiva                 26
Valledupar            24
Florencia             23
Girardot              23
Pereira               19
Honda                 16
Riohacha              15
Barranquilla          14
Santa Marta           12
Cúcuta                11
Dos Quebradas         10
Montería               8
Medellín               5
Tabatinga              3
Los Olivos             2
El Espinal             2
Aldana                 1
Cajibio                1
Tello                  1
Buritaca               1
Villamarín             1
Bogotá                 1
Socorro                1
Melgar                 1
Soledad                1
Tocaima                1
Soacha                 1
Fonseca                1


### Question 13

In [25]:
def fix_13(x):
    if x == 'Cada Semana':
        r = 'Cada semana'
    elif x == 'Una vez al mes' or x == 'Cada tres semanas' or x == 'Cada dos meses':
        r = x
    elif x == 'Otra: Cada dos semanas':
        r = 'Cada dos semanas'
    else:
        r = 'OTHER'
    return r

survey_df[13] = survey_df[13].apply(fix_13)

In [26]:
survey_df[13].value_counts()

Cada semana          256
Una vez al mes       112
Cada tres semanas     93
Cada dos meses        63
OTHER                 45
Cada dos semanas      21
Name: 13, dtype: int64

### Question 16

In [28]:
def fix_16(x):
    if x is np.nan:
        aux = ['UNKNOWN']
    elif '*' in x:
        aux = [y.strip().lower() for y in x.split('*')[1:]]
    else:
        aux = [x.strip().lower()]
    r = []
    for y in aux:
        if y == 'descansar' or y == 'leer' or y == 'estudiar' or y == 'pasear':
            r.append(y)
        elif y == 'otra: trabajar' or y == 'otra. trabajar':
            r.append('trabajar')
        elif 'orar' in y or 'oración' in y or 'catolicos' in y or 'iglesia' in y or 'misa' in y:
            r.append('iglesia')
        else:
            r.append('OTHER')
    return r

survey_df[16] = survey_df[16].apply(fix_16)

In [29]:
q16 = [y for x in survey_df[16] for y in x]

In [30]:
c16 = Counter(q16)

In [31]:
c16.most_common()

[('descansar', 309),
 ('leer', 308),
 ('estudiar', 165),
 ('pasear', 126),
 ('OTHER', 37),
 ('trabajar', 17),
 ('iglesia', 13)]

### Question 17

In [37]:
def fix_17(x):
    if x is np.nan:
        aux = ['UNKNOWN']
    elif '*' in x:
        aux = [y.strip().lower() for y in x.split('*')[1:]]
    else:
        aux = [x.strip().lower()]
    r = []
    for y in aux:
        if y == 'hacer trabajos academicos' or \
           y == 'para leer por gusto personal' or \
           y == 'como soporte para integración':
            r.append(y)
        elif y == 'como soporte para integracion':
            r.append('como soporte para integración')
        else:
            r.append('OTHER')
    return r

survey_df[17] = survey_df[17].apply(fix_17)

In [38]:
q17 = [y for x in survey_df[17] for y in x]

In [39]:
c17 = Counter(q17)

In [40]:
c17.most_common()

[('hacer trabajos academicos', 340),
 ('para leer por gusto personal', 316),
 ('OTHER', 51),
 ('como soporte para integración', 47)]

### Question 19

In [49]:
def fix_19(x):
    if x is np.nan:
        aux = ['UNKNOWN']
    elif '*' in x:
        aux = [y.strip().lower() for y in x.split('*')[1:]]
    else:
        aux = [x.strip().lower()]
    r = []
    for y in aux:
        if y == 'no tuvo tiempo la semana pasada' or \
           y == 'no acostumbra hacer uso de este servicio durante la semana santa' or \
           y == 'no sabía que lunes, martes y miercoles santos se prestó este servicio en forma normal' or \
           y == 'salio de viaje' or \
           y == 'no queria comprometerse con el préstamo de material documental estos dias':
            r.append(y)
        else:
            r.append('OTHER')
    return r

survey_df[19] = survey_df[19].apply(fix_19)

In [50]:
q19 = [y for x in survey_df[19] for y in x]

In [51]:
c19 = Counter(q19)

In [52]:
c19.most_common()

[('OTHER', 171),
 ('no tuvo tiempo la semana pasada', 139),
 ('no acostumbra hacer uso de este servicio durante la semana santa', 92),
 ('no sabía que lunes, martes y miercoles santos se prestó este servicio en forma normal',
  85),
 ('salio de viaje', 76),
 ('no queria comprometerse con el préstamo de material documental estos dias',
  39)]

### Subset

In [ ]:
# survey_df = survey_df[
#     ['SUCURSAL', 'SEXO', 'RANGO_EDAD', 'NIVEL_ESCOLAR_FINALIZADO', 'DEPARTAMENTO'] + range
# ]